Rebuild street graph
====================
Loads a street graph and reallocates roads space, e.g., into a network of one-way streets

In [1]:
import snman
from snman import osmnx_customized as oxc
from snman.constants import *
import geopandas as gpd

PERIMETER = '_debug'
# Set SAVE_TO_DEBUG = True for saving the results into the _debug folder
# which is automatically used in the QGIS files
SAVE_TO_DEBUG = True

# Set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data_v2/'
inputs_path = data_directory + 'inputs/'
process_path = data_directory + 'process/' + PERIMETER + '/'

if SAVE_TO_DEBUG:
    export_path = data_directory + 'outputs/' + '_debug' + '/'
else:
    export_path = data_directory + 'outputs/' + PERIMETER + '/'

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
# =====================================================================================
# LOAD DATA
# =====================================================================================

print('Load street graph')
G = snman.io.load_street_graph(process_path + 'street_graph_edges.gpkg', process_path + 'street_graph_nodes.gpkg')

Load street graph


In [3]:
print('Load perimeters')
perimeters_gdf = snman.io.load_perimeters(inputs_path + 'perimeters/perimeters.shp')

print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    inputs_path + 'rebuilding_regions/rebuilding_regions.gpkg'
)

print('Load measurement regions')
# Polygons that define areas where network measures will be calculated
measurement_regions_gdf = snman.io.load_measurement_regions(
    inputs_path + 'measurement_regions/measurement_regions.gpkg'
)

Load perimeters
Load rebuilding regions
Load measurement regions


In [4]:
print('Load POIs')
poi_gpd = snman.io.load_poi(inputs_path + 'poi/poi.gpkg').clip(perimeters_gdf.loc['zrh_north-west'].geometry)

Load POIs


In [5]:
# ---------------------------
# rebuild regions
# ---------------------------

import networkx as nx

# initialize lanes after rebuild
nx.set_edge_attributes(G, nx.get_edge_attributes(G, KEY_LANES_DESCRIPTION), KEY_LANES_DESCRIPTION_AFTER)

for i, rebuilding_region in rebuilding_regions_gdf.iterrows():

    hierarchies_to_fix = snman.hierarchy.HIERARCHIES.difference(rebuilding_region['hierarchies_to_include']).union(rebuilding_region['hierarchies_to_fix'])

    polygon = rebuilding_region['geometry']
    # make subgraph
    H = oxc.truncate.truncate_graph_polygon(G, polygon, quadrat_width=100, retain_all=True)
    # keep only hierarchies toi include
    if len(rebuilding_region['hierarchies_to_include']) > 0:
        H = snman.street_graph.filter_by_hierarchy(H, rebuilding_region['hierarchies_to_include'])
    # set given lanes according to network rules
    snman.distribution.set_given_lanes(
        H,
        maintain_motorized_access_on_street_level=rebuilding_region['keep_all_streets'],
        hierarchies_to_fix=rebuilding_region['hierarchies_to_fix']
    )
    # get only the car lanes
    H = snman.street_graph.filter_by_modes(H, {MODE_PRIVATE_CARS})
    # make lane graph
    L = snman.street_graph.to_lane_graph(H, KEY_GIVEN_LANES_DESCRIPTION)

    snman.io.export_street_graph(H, export_path + 'H_edges.gpkg', export_path + 'H_nodes.gpkg')
    snman.io.export_street_graph(L, export_path + 'L_edges.gpkg', export_path + 'L_nodes.gpkg')
    L = snman.graph_utils.keep_only_the_largest_connected_component(L)

    # link elimination
    L = snman.owtop.link_elimination(L, verbose=True)
    #snman.io.export_street_graph(L, export_path + 'L_edges.gpkg', export_path + 'L_nodes.gpkg')
    # rebuild lanes in subgraph based on new lane graph
    snman.owtop.rebuild_lanes_from_owtop_graph(
        H,
        L,
        hierarchies_to_protect=rebuilding_region['hierarchies_to_fix'],
        source_lanes_attribute=KEY_LANES_DESCRIPTION_AFTER,
        target_lanes_attribute=KEY_LANES_DESCRIPTION_AFTER
    )
    # write rebuilt lanes from subgraph into the main graph
    nx.set_edge_attributes(G, nx.get_edge_attributes(H, KEY_LANES_DESCRIPTION_AFTER), KEY_LANES_DESCRIPTION_AFTER)

Initialized graph has  222  nodes and  491  edges
Iteration  10
Iteration  20
Iteration  30
Iteration  40
Iteration  50
Iteration  60
Iteration  70
Iteration  80
Iteration  90
Iteration  100
Iteration  110
Iteration  120
Iteration  130
Iteration  140
Iteration  150
Iteration  160
Iteration  170
Iteration  180
Iteration  190
Iteration  200
Iteration  210
Iteration  220
Iteration  230
Initialized graph has  121  nodes and  345  edges
Iteration  10
Iteration  20
Iteration  30
Iteration  40
Iteration  50
Iteration  60
Iteration  70
Iteration  80
Iteration  90
Iteration  100
Iteration  110
Iteration  120
Iteration  130
Iteration  140
Iteration  150
Initialized graph has  265  nodes and  709  edges
Iteration  10
Iteration  20
Iteration  30
Iteration  40
Iteration  50
Iteration  60
Iteration  70
Iteration  80
Iteration  90
Iteration  100
Iteration  110
Iteration  120
Iteration  130
Iteration  140
Iteration  150
Iteration  160
Iteration  170
Iteration  180
Iteration  190
Iteration  200
Iterati

In [6]:
snman.io.export_street_graph(L, export_path + 'L_edges.gpkg', export_path + 'L_nodes.gpkg')

In [7]:
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

In [11]:
# =====================================================================================
# EXPORT
# =====================================================================================

if 1:
    print('Export network without lanes')
    snman.io.export_street_graph(G, export_path + 'street_graph_edges.gpkg', export_path + 'street_graph_nodes.gpkg')

if 1:
    print('Export network with lanes')
    SCALING = 4
    snman.io.export_street_graph_with_lanes(G, KEY_LANES_DESCRIPTION, export_path + 'lane_geometries.shp', scaling=SCALING)
    snman.io.export_street_graph_with_lanes(G, KEY_LANES_DESCRIPTION_AFTER, export_path + 'lane_geometries_after.shp', scaling=SCALING)

if 0:
    print('Export OSM XML')
    snman.io.export_osm_xml(G, export_path + 'new_network.osm', {
        'highway', 'lanes', 'lanes:forward', 'lanes:backward', 'lanes:both_ways',
        'cycleway', 'cycleway:lane', 'cycleway:left', 'cycleway:left:lane', 'cycleway:right', 'cycleway:right:lane',
        'bus:lanes:backward', 'bus:lanes:forward', 'vehicle:lanes:backward', 'vehicle:lanes:forward',
        'maxspeed', 'oneway',
        '_connected_component'
    }, uv_tags=True, tag_all_nodes=True)

Export network without lanes
Export network with lanes


In [16]:
snman.io.export_street_graph(snman.street_graph.filter_by_modes(
    G, {MODE_PRIVATE_CARS}, lane_description_key=KEY_LANES_DESCRIPTION_AFTER), export_path + 'H_edges.gpkg', export_path + 'H_nodes.gpkg'
)
